In [15]:
import pandas as pd
import numpy as np
import os
import glob

pd.options.display.max_columns = None
pd.options.display.max_rows = None

#os.chdir('/MFMDatasets/MFM_bopf/src/multi_predict/SVC/output')
#os.chdir('/slurm_storage/mbopf/projects/MFM_bopf/src/multi_predict/SVC/output')
os.chdir('/slurm_storage/mbopf/projects/MFM_bopf/src/multi_predict/MLP/output')
#files = glob.glob("*-NB-*.out")
#files = glob.glob("*-LR-*.out")
#files = glob.glob("*-RF-*.out")
#files = glob.glob("*-SVC-*.out")
#files = glob.glob("*-LSVC-*.out")
#files = glob.glob("*-MLP-*.out")

#files = glob.glob("*high_Age-MLP-*.out")
#ROC_CUTOFF = 0.795
#ROC_CUTOFF = 0.80
ROC_CUTOFF = 0.5
#files = glob.glob("*-MLP-*.out")
files = glob.glob("transfus_yes*-MLP-*.out")

In [16]:
files.sort()
for i in range(len(files)-1):
    if files[i][:-20] == files[i+1][:-20]:
        print(f'{files[i]}')
    

transfus_yes-1-RAND-MLP-relu_1e-05_[52,26,13,6]_0.001_500_7_adam-1-1.0-20201023_140150.out
transfus_yes-1-RAND-MLP-relu_1e-05_[64,32,16,8,4]_0.001_500_7_adam-1-1.0-20201023_140141.out


In [12]:
from tokenize import tokenize

pd.options.display.float_format = '{:,.4f}'.format
files.sort()
prev_f = ""
coeffs_df = pd.DataFrame()
coeffs_rank_df = pd.DataFrame()
for idx, f in enumerate(files):
    if prev_f[:-20] == f[:-20]:   #if the filenames only differ by timestamp
        print(f'{prev_f}')
        continue   #skip "duplicate"
 
    with open(f, 'r') as file:
        found_coef = False
        cutoff = False
        coef_series = pd.Series()
        ind_vars = []
        coef_vals = []
        for line in file:
            # Add ROC_AUC cutoff
            if "ROC_AUC" in line:
                roc_auc = line.split()[2]   
                if float(roc_auc) < ROC_CUTOFF:
                    #print(f'roc_auc = {roc_auc}')
                    cutoff = True
                    break    # Stop processing this file; poor performance
            
            if "coeffs =" in line:
                found_coef = True
                continue
            
            if found_coef:
                var_coef = line.split()
                if var_coef[0] != 'dtype:':
                    ind_vars.append(var_coef[0])
                    coef_vals.append(float(var_coef[1]))
                
        if not cutoff:
            if coeffs_df.empty:
                coeffs_df = pd.DataFrame(data=coef_vals, index=ind_vars, columns=[idx])
                coeffs_rank_df = pd.DataFrame(data=coeffs_df[idx].rank(ascending=False).astype(int), columns=[idx])
            else:
                this_df = pd.DataFrame(data=coef_vals, index=ind_vars, columns=[idx])
                this_rank_df = pd.DataFrame(data=this_df[idx].rank(ascending=False).astype(int), columns=[idx])
                coeffs_df = coeffs_df.merge(this_df, left_index=True, right_index=True, how="right")
                coeffs_rank_df = coeffs_rank_df.merge(this_rank_df, left_index=True, right_index=True, how="right")

    prev_f = f
    
#if 'high_Age' in coeffs_df.index:
#    coeffs_df.drop('high_Age', inplace=True)
#    coeffs_rank_df.drop('high_Age', inplace=True)
#if 'Insurance' in coeffs_df.index:
#    coeffs_df.drop('Insurance', inplace=True)
#    coeffs_rank_df.drop('Insurance', inplace=True)
coeffs_df

,0,1,2,3,4,5,6,7,8,9,10,11,13,14,16,17,18,19,20,21,22,24,25,26,27,29,31,35,36,37,38,39,40,41,43,44,45,46,47,48,49,50,51,53,54,55,56,57,59,61,62,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,88,89,90,91,93,95,96,99,100,101,102,103,104,105,106,107,109,110,111,112,113,115,116,117,120,121,122,123,124,125,126,127,128,130,131,133,134,136,137,138,140,141,142,143,144,145,146,147,148,149,150,151,152,153
abruptio9,0.2462,0.4793,0.6770,0.0689,0.0579,0.0418,0.3194,0.4148,0.7226,0.2100,0.5226,0.7331,0.0831,0.0596,0.2819,0.8545,0.5964,0.3352,0.5167,0.1671,0.2985,0.0993,0.1800,0.4686,0.5929,0.1251,0.6538,0.5796,0.3131,0.2008,0.2594,0.1118,0.0709,0.0738,0.1017,0.0766,0.0534,0.0776,0.0884,0.0434,0.0830,0.0425,0.0614,0.0462,0.0363,0.0369,0.0217,0.0274,0.0310,0.0277,0.0165,0.0203,0.0203,0.0143,0.0194,0.0159,0.0133,0.0082,0.0135,0.0089,0.0102,0.0088,0.0069,0.0039,0.0077,0.0083,0.0079,0.0062,0.0062,0.0059,0.0054,0.0079,0.1710,0.3133,0.0993,0.1830,0.6178,0.7141,0.1251,0.6453,0.7767,0.3164,0.6133,0.3748,0.5533,0.5003,0.2016,0.4959,0.1753,0.1851,0.0991,0.2928,0.4526,0.3674,0.1243,0.6499,0.4995,0.7775,0.3927,0.3234,0.1683,0.3123,0.0829,0.0904,0.0609,0.1829,0.1938,0.0991,0.1391,0.6487,0.1242,0.6491,0.6022,0.9896,0.1098,0.4079,0.5233,0.2010,0.5270,0.1921,0.4607,0.7434,0.0650,0.0597,0.0586,0.2001,0.5010,0.6857
Delmode,0.1295,0.2170,0.3017,0.1178,0.1661,0.1627,0.1440,0.2139,0.2996,0.1139,0.2251,0.2982,0.1597,0.1948,0.1905,0.2993,0.2866,0.2468,0.2797,0.1092,0.1537,0.1013,0.1246,0.2518,0.3880,0.1288,0.3189,0.3033,0.2652,0.2311,0.2599,0.2341,0.2208,0.2524,0.2277,0.2783,0.1245,0.1660,0.1603,0.2006,0.1899,0.2032,0.2334,0.1965,0.2397,0.1688,0.1693,0.1890,0.1705,0.1397,0.1464,0.1634,0.1425,0.1654,0.1178,0.1212,0.1217,0.0507,0.1146,0.1346,0.0870,0.0934,0.0767,0.0293,0.0692,0.0706,0.0740,0.0764,0.0529,0.0524,0.0627,0.0605,0.1120,0.1551,0.1062,0.1247,0.2767,0.3859,0.1284,0.3176,0.3197,0.2097,0.2862,0.2198,0.2687,0.2637,0.1367,0.2183,0.1087,0.1024,0.0980,0.1470,0.2558,0.2168,0.1375,0.2804,0.2935,0.3391,0.1840,0.2388,0.1322,0.2040,0.0914,0.1898,0.2144,0.1107,0.1069,0.0980,0.0934,0.2701,0.1360,0.2689,0.2854,0.3458,0.1128,0.1798,0.2643,0.1390,0.2289,0.1162,0.2507,0.2965,0.0935,0.1846,0.2053,0.1147,0.2401,0.2879
Anteanemia,0.0944,0.2426,0.3856,0.0788,0.1380,0.1201,0.1172,0.1891,0.3009,0.0896,0.2281,0.3213,0.0837,0.0870,0.1241,0.3167,0.2170,0.1451,0.2142,0.0960,0.1398,0.0942,0.0730,0.2047,0.4715,0.1321,0.3577,0.2562,0.1920,0.1296,0.1789,0.1672,0.1339,0.1894,0.1144,0.1301,0.0824,0.0668,0.1113,0.1118,0.1304,0.1265,0.1243,0.1215,0.0750,0.0753,0.0602,0.0733,0.0686,0.0690,0.0469,0.0534,0.0582,0.0350,0.0489,0.0409,0.0385,0.0203,0.0392,0.0236,0.0293,0.0238,0.0199,0.0092,0.0215,0.0212,0.0212,0.0144,0.0158,0.0132,0.0136,0.0135,0.0993,0.1426,0.0945,0.0767,0.2450,0.3921,0.1322,0.2690,0.2776,0.1404,0.2198,0.1362,0.2138,0.2483,0.1387,0.1971,0.0876,0.0915,0.0941,0.1331,0.1449,0.1352,0.1309,0.3235,0.2123,0.4079,0.1528,0.1976,0.1325,0.1762,0.0668,0.1448,0.1124,0.0937,0.0959,0.0941,0.0759,0.2738,0.1319,0.2347,0.2188,0.5209,0.1268,0.1558,0.2498,0.1434,0.1987,0.0951,0.2071,0.3848,0.0647,0.1185,0.1192,0.0979,0.2144,0.2675
IntraMgSO,0.0985,0.1372,0.2067,0.0634,0.0734,0.0550,0.1058,0.1280,0.1998,0.0996,0.1862,0.2247,0.0677,0.0711,0.1030,0.2297,0.1508,0.1416,0.1801,0.0815,0.1160,0.1041,0.0845,0.1795,0.2322,0.1259,0.2267,0.1640,0.1361,0.1230,0.1456,0.0956,0.0938,0.1027,0.0985,0.0756,0.0573,0.0746,0.0751,0.0707,0.0895,0.0786,0.0889,0.0702,0.0502,0.0482,0.0490,0.0479,0.0488,0.0355,0.0351,0.0337,0.0330,0.0236,0.0284,0.0271,0.0237,0.0144,0.0233,0.0177,0.0161,0.0158,0.0134,0.0040,0.0127,0.0139,0.0121,0.0109,0.0113,0.0098,0.0091,0.0099,0.0850,0.1172,0.1042,0.0858,0.1861,0.2121,0.1259,0.1953,0.2053,0.1412,0.1552,0.1438,0.1882,0.1895,0.1346,0.1727,0.0862,0.0940,0.1040,0.1065,0.1546,0.1479,0.1256,0.2321,0.1762,0.2627,0.1392,0.1511,0.1286,0.1499,0.0691,0.1019,0.0821,0.0915,0.0976,0.1040,0.0807,0.1854,0.1261,0.2022,0.1735,0.3132,0.1213,0.1389,0.1932,0.1392,0.1809,0.0940,0.1552,0.261

In [13]:
coeffs_rank_df['mean_rank'] = coeffs_rank_df.apply(np.mean, axis=1).rank().astype(int)
coeffs_rank_df = coeffs_rank_df.astype('Int8')
coeffs_rank_df.sort_values('mean_rank', inplace=True)
coeffs_rank_df

,0,1,2,3,4,5,6,7,8,9,10,11,13,14,16,17,18,19,20,21,22,24,25,26,27,29,31,35,36,37,38,39,40,41,43,44,45,46,47,48,49,50,51,53,54,55,56,57,59,61,62,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,88,89,90,91,93,95,96,99,100,101,102,103,104,105,106,107,109,110,111,112,113,115,116,117,120,121,122,123,124,125,126,127,128,130,131,133,134,136,137,138,140,141,142,143,144,145,146,147,148,149,150,151,152,153,mean_rank
Delmode,2,5,4,1,3,2,2,2,4,2,6,4,1,1,2,4,2,2,2,3,3,27,2,2,3,17,3,2,2,1,1,1,1,1,1,1,1,2,2,1,2,1,1,2,1,1,1,1,1,2,2,1,2,1,2,1,1,1,2,1,2,2,2,3,2,2,2,2,2,1,2,2,2,3,15,2,3,3,17,2,2,2,2,2,2,3,4,4,2,4,35,4,2,2,6,7,2,4,4,3,3,2,1,1,1,3,4,35,3,5,6,4,2,5,31,4,3,5,3,2,2,4,1,1,1,2,2,2,1
Intratocolytix,4,4,11,3,2,3,6,4,8,5,8,6,2,2,3,6,5,3,3,2,5,4,3,3,7,3,4,5,3,2,3,2,2,2,2,2,2,1,1,2,1,2,2,1,2,2,2,2,2,1,1,2,1,2,1,2,2,2,1,2,1,1,1,1,1,1,1,1,1,2,1,1,3,6,2,3,2,5,3,3,4,3,4,3,3,6,9,5,3,7,20,8,3,4,3,13,4,12,3,4,8,4,3,2,2,2,7,20,7,7,5,8,4,15,1,3,6,8,5,5,6,8,3,2,2,4,6,6,2
momrace_new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,7,7,23,6,4,20,8,9,3
Anteanemia,29,3,3,23,4,4,7,5,3,27,5,2,25,14,9,2,6,5,4,13,4,42,38,5,2,12,2,4,5,5,4,3,4,3,5,5,10,43,4,4,4,3,3,4,18,19,19,13,19,13,22,15,15,21,16,17,15,5,16,24,16,19,17,19,18,19,17,23,18,16,20,21,11,4,41,25,5,2,12,4,3,5,5,8,4,4,3,6,26,26,42,5,11,11,12,4,6,2,5,5,2,5,38,3,4,20,21,42,25,4,11,5,6,2,5,5,4,3,6,19,5,2,36,5,3,15,4,3,4
HosEpiNurse,16,8,5,24,18,6,5,9,5,36,4,11,26,35,7,10,4,9,6,6,6,34,14,4,4,48,6,12,4,8,5,4,5,4,25,3,16,27,3,11,5,4,6,19,10,24,20,17,29,19,19,18,25,20,18,19,22,8,24,20,18,22,23,48,20,20,22,25,22,28,31,24,5,5,34,12,4,4,48,6,6,14,3,10,8,5,7,7,5,27,34,7,4,5,47,6,11,6,6,6,7,7,36,4,23,6,20,34,32,6,49,7,16,6,36,6,5,7,7,21,3,5,34,3,20,22,3,7,5
Education,30,21,15,17,13,12,35,14,14,19,24,19,12,5,10,20,29,42,26,31,36,9,7,19,21,2,23,31,20,21,19,16,15,6,3,4,19,13,15,24,3,6,4,3,3,12,13,12,12,12,12,12,12,3,13,12,11,3,12,9,13,11,10,4,10,5,12,5,9,6,10,6,37,38,3,8,22,24,2,17,16,12,26,35,23,17,24,29,33,25,15,26,12,14,2,28,16,25,27,12,19,25,10,16,8,36,25,14,6,31,4,24,12,25,9,26,21,28,29,23,20,29,14,16,15,26,23,25,6
abruptio9,1,1,1,27,39,44,1,1,1,1,1,1,27,30,1,1,1,1,1,1,1,32,1,1,1,25,1,1,1,3,2,10,37,40,17,32,38,32,17,41,23,43,34,43,41,41,45,44,45,40,45,42,43,44,40,42,42,41,42,44,42,41,44,35,41,41,42,41,41,37,43,35,1,1,32,1,1,1,25,1,1,1,1,1,1,1,1,1,1,1,31,1,1,1,26,1,1,1,1,1,1,1,11,23,34,1,1,31,1,1,23,1,1,1,40,1,1,1,1,1,1,1,33,38,34,1,1,1,7
abruption,11,9,8,30,23,25,11,7,7,14,9,10,29,29,5,8,13,7,9,15,20,26,13,9,18,34,17,11,11,10,6,19,23,8,31,25,35,48,29,20,24,18,21,29,30,27,28,23,31,26,27,24,27,29,24,25,25,15,28,32,25,28,26,26,24,22,26,26,25,25,28,23,13,20,27,14,10,14,34,12,14,6,10,5,9,12,5,9,11,23,27,15,10,8,31,9,12,7,8,13,6,11,43,8,17,12,27,27,27,8,31,10,8,10,2,7,12,6,9,6,9,10,31,25,22,9,9,11,8
Marital,8,11,13,8,24,22,9,13,26,26,20,21,31,25,29,21,7,20,15,36,10,28,48,17,9,11,8,7,6,12,13,7,12,5,26,6,6,40,28,21,17,9,22,14,14,29,23,21,25,14,25,20,19,15,20,27,18,4,20,15,20,21,18,24,17,15,24,19,17,10,18,14,31,11,28,49,9,8,11,11,8,20,7,33,15,7,14,14,48,31,28,12,9,17,10,8,9,15,12,7,20,12,25,9,29,47,33,28,48,19,10,12,18,12,33,11,7,18,15,32,13,13,19,21,24,34,13,17,9
IntraMgSO,19,18,10,38,30,34,17,21,10,6,10,7,37,26,39,7,18,8,5,49,13,18,6,7,11,23,9,15,14,9,7,31,26,22,22,33,34,38,27,28,16,22,14,26,32,35,27,26,28,35,30,27,30,30,31,28,24,21,31,27,27,29,28,33,32,26,29,27,23,22,30,26,46,12,19,6,8,11,22,7,10,4,16,4,5,8,6,8,31,14,16,10,7,7,22,12,10,8,9,9,5,10,32,10,22,26,14,16,9,11,19,9,9,7,12,9,9,4,8,22,12,6,35,24,18,10,10,4,10


In [5]:
#coeffs_rank_df.to_csv('/MFMDatasets/MFM_bopf/src/multi_predict/MLP/output/MLPcoef_rank.csv')